# Applied Topological Data Analysis: a practical example.
How to use Ripser and Giotto-TDA basic algorithms.

Another fancy tutorial is:
https://giotto-ai.github.io/gtda-docs/0.5.1/notebooks/vietoris_rips_quickstart.html

The idea of this hands-on tutorial is to create a sample of a known manifold (in this example, a torus) and to compute its three first persistent diagrams (dimensions $0$, $1$ and $2$).

Therefore, the first action we must perform is to generate our torus point cloud sample. We will use its 3D parametrization to generate its points. First, we will generate a random (uniform) sample  of $[\sqrt{n}]$ values for each of its parameters. Then, we will generate the points of the manifold for each possible combination of values ($n$) (the torus has $2$ parameters). At the end, we will add some noise to this points to simulate realistic environmental measures.

In [143]:
import numpy as np
import random

In [144]:
def _generate_random_list_of_points(min_number, max_number, number_of_elements):
    return [random.uniform(min_number, max_number) for _ in range(number_of_elements)]

# Make point clouds for tori. n_points should be a square power.
def get_sample_of_torus(n_points: int, noise: float):
    number_of_points_per_list = int(np.sqrt(n_points))
    torus_point_clouds = []
    s_range = _generate_random_list_of_points(0, 2*np.pi, number_of_points_per_list)
    t_range = _generate_random_list_of_points(0, 2*np.pi, number_of_points_per_list)
    torus_point_cloud = np.asarray([
    [
        (2 + np.cos(s)) * np.cos(t) + noise * (np.random.rand(1)[0] - 0.5),
        (2 + np.cos(s)) * np.sin(t) + noise * (np.random.rand(1)[0] - 0.5),
        np.sin(s) + noise * (np.random.rand(1)[0] - 0.5),
    ]
    for t in s_range
    for s in t_range
])  
    return torus_point_cloud

In [147]:
# Now we generate the torus point cloud.
torus_point_cloud = get_sample_of_torus(900, 0.1)

In [148]:
from gtda.plotting import plot_point_cloud

Now, let's visualize our torus point cloud, to see that everything is in place.

In [116]:
plot_point_cloud(torus_point_cloud)

Now, **let's compute persistent homology!**
(using Ripser and Giotto-tda)

In [117]:
from gtda.homology import VietorisRipsPersistence
from gtda.plotting import plot_diagram
import time

Ripser in Giotto-TDA only needs few parameters to work. It follows the standards of Scikit learn so we will need first, to create a`VietorisRipsPersistence` object and then use the `fit_transform` function to obtain the persistence diagram.

In [149]:
def compute_persistence_diagram(point_cloud: np.array, 
                                collapse:bool=False):
    start_t = time.time()
    VR = VietorisRipsPersistence(metric='euclidean', 
                                 homology_dimensions=(0,1,2),
                                coeff=2,
                                collapse_edges=collapse)
    pd = VR.fit_transform([point_cloud])
    end_t = time.time()
    print(f'Persistence diagrams computated in {end_t - start_t} s.')
    return pd

Here, we execute VR with the usual parameters.

In [119]:
torus_pd = compute_persistence_diagram(torus_point_cloud)[0]

Persistence diagrams computated in 68.07980132102966 s.


Here, we execute Vietoris-Rips preprocessing the point cloud using the algorithm edge collapse [1].

[1] J.-D. Boissonnat and S. Pritam, “Edge Collapse and Persistence of Flag Complexes”; in 36th International Symposium on Computational Geometry (SoCG 2020), pp. 19:1–19:15, Schloss Dagstuhl-Leibniz–Zentrum für Informatik, 2020; DOI: 10.4230/LIPIcs.SoCG.2020.19.

In [120]:
torus_pd_with_collapse = compute_persistence_diagram(torus_point_cloud, 
                                                     True)[0]

Persistence diagrams computated in 30.931909799575806 s.


Let's visualize the persistence diagrams for both computations, we'll see that they are **almost equal!**

In [121]:
plot_diagram(torus_pd)

In [122]:
plot_diagram(torus_pd_with_collapse)

Now we'll compute the persistence diagram without noise, using the confidence set algorithm published in the paper ["Confidence sets for persistence diagrams"](https://projecteuclid.org/journals/annals-of-statistics/volume-42/issue-6/Confidence-sets-for-persistence-diagrams/10.1214/14-AOS1252.full), by Fasy et al.

In [1]:
# from math import comb
# You need numba for this package
import sys
!{sys.executable} -m pip install numba
from hausdorff import hausdorff_distance

"c:\users\rub‚n" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


ModuleNotFoundError: No module named 'hausdorff'

In [135]:
def L(t, original_point_cloud: np.array, subsamples:list[np.array]):
    T_distances = [hausdorff_distance(original_point_cloud, subsample)
                  for subsample in subsamples]
    T_distances_geq_t = list(
        filter(lambda dist: dist > t, T_distances)
    )
    return (1/float((len(subsamples))))*np.sum(T_distances_geq_t)


# N = comb(m, b) -> Unfeasible in practice
def compute_confidence_set(point_cloud: np.array, alpha: float, N=1000):
    m = point_cloud.shape[0]
    b = int(m/np.log(m))
    subsamples = []
    for _ in range(N):
        indices = [random.randint(0, m-1) for _ in range(b)]
        sampled_point_cloud = point_cloud[indices]
        subsamples.append(sampled_point_cloud)
    c_alpha = 1/(float(L(alpha, point_cloud, subsamples)))
    return c_alpha

In [136]:
c_alpha = compute_confidence_set(torus_point_cloud, 0.2)

In [137]:
def filter_persistence_diagram_by_confidence_set(pd: np.array, c_alpha):
    notable_points = []
    for point in pd:
        x, y = point[0] + c_alpha, point[1] + c_alpha
        if x <= point[1]:
            notable_points.append(point)
    return np.vstack(notable_points)

In [138]:
torus_pd_filt = filter_persistence_diagram_by_confidence_set(
    torus_pd, c_alpha)
torus_pd_with_coll_filt = filter_persistence_diagram_by_confidence_set(
    torus_pd_with_collapse, c_alpha)

In [139]:
plot_diagram(torus_pd_filt)

In [140]:
plot_diagram(torus_pd_with_coll_filt)